In [1]:
import gym
import time
import torch
import argparse
import numpy as np

from logger import Logger

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation

from copy import deepcopy
from argparse import Namespace

from option_critic import OptionCriticFeatures, OptionCriticConv
from option_critic import critic_loss as critic_loss_fn
from option_critic import actor_loss as actor_loss_fn

from train import run
from logger import Logger
from utils import to_tensor
from fourrooms import Fourrooms
from environment import make_env
from experience_replay import ReplayBuffer

2023-07-04 12:59:27.278394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 12:59:27.840254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Parameters

In [2]:
ENV = "ltl_fourrooms"

In [3]:
env, is_atari = make_env("ltl_fourrooms", False)

In [4]:
env.spec.specs[0].ldba.delta

[{(): 0, ('phi',): 1, ('psi',): 0, ('phi', 'psi'): 1},
 {(): 2, ('phi',): 2, ('psi',): 0, ('phi', 'psi'): 1},
 {(): 2, ('phi',): 2, ('psi',): 2, ('phi', 'psi'): 2}]

In [5]:
env.spec.specs[0].ldba.state

0

In [6]:
env.spec.specs[0].ldba.reset()

(0, False)

In [7]:
env.spec.specs[0].ldba.step(('phi',))

(1, True)

In [8]:
env.spec.specs[0].ldba.acc

[{(): [None], ('phi',): [True], ('psi',): [True], ('phi', 'psi'): [True]},
 {(): [None], ('phi',): [None], ('psi',): [True], ('phi', 'psi'): [True]},
 {(): [None], ('phi',): [None], ('psi',): [None], ('phi', 'psi'): [None]}]

## Training

In [9]:
args = Namespace(batch_size=32, cuda=True, entropy_reg=0.01, env='ltl_fourrooms', 
                 epsilon_decay=20000, epsilon_min=0.1, epsilon_start=1.0, exp=None, 
                 frame_skip=4, freeze_interval=200, gamma=0.99, learning_rate=0.0005, 
                 logdir='runs', max_history=10000, max_steps_ep=18000, max_steps_total=4000000, 
                 num_options=6, optimal_eps=0.05, seed=0, switch_goal=True, temp=1, 
                 termination_reg=0.01, update_frequency=4, render=False, name="Standard")
# logger = Logger(
#         logdir=args.logdir, 
#         run_name=f"{OptionCriticFeatures.__name__}-{args.env}-{args.exp}-{time.ctime()}")

run(args)

Current goal {16: ('phi',), 17: ('psi',)}


2023/03/20 11:31:36 PM > ep 1 done. total_steps=136 | reward=5.0 | episode_steps=136 | hours=0.000 | epsilon=0.994
2023/03/20 11:31:39 PM > ep 2 done. total_steps=1136 | reward=0 | episode_steps=1000 | hours=0.001 | epsilon=0.950
2023/03/20 11:31:41 PM > ep 3 done. total_steps=1713 | reward=8.0 | episode_steps=577 | hours=0.002 | epsilon=0.926
2023/03/20 11:31:41 PM > ep 4 done. total_steps=1728 | reward=1.0 | episode_steps=15 | hours=0.002 | epsilon=0.926
2023/03/20 11:31:42 PM > ep 5 done. total_steps=1824 | reward=2.0 | episode_steps=96 | hours=0.002 | epsilon=0.922
2023/03/20 11:31:42 PM > ep 6 done. total_steps=1899 | reward=3.0 | episode_steps=75 | hours=0.002 | epsilon=0.919
2023/03/20 11:31:43 PM > ep 7 done. total_steps=2349 | reward=2.0 | episode_steps=450 | hours=0.002 | epsilon=0.900
2023/03/20 11:31:44 PM > ep 8 done. total_steps=2543 | reward=5.0 | episode_steps=194 | hours=0.002 | epsilon=0.893
2023/03/20 11:31:47 PM > ep 9 done. total_steps=3428 | reward=4.0 | episode_s

2023/03/20 11:32:46 PM > ep 71 done. total_steps=21470 | reward=5.0 | episode_steps=109 | hours=0.020 | epsilon=0.408
2023/03/20 11:32:46 PM > ep 72 done. total_steps=21630 | reward=3.0 | episode_steps=160 | hours=0.020 | epsilon=0.405
2023/03/20 11:32:47 PM > ep 73 done. total_steps=21869 | reward=2.0 | episode_steps=239 | hours=0.020 | epsilon=0.402
2023/03/20 11:32:47 PM > ep 74 done. total_steps=22029 | reward=2.0 | episode_steps=160 | hours=0.020 | epsilon=0.399
2023/03/20 11:32:48 PM > ep 75 done. total_steps=22151 | reward=1.0 | episode_steps=122 | hours=0.020 | epsilon=0.397
2023/03/20 11:32:48 PM > ep 76 done. total_steps=22259 | reward=3.0 | episode_steps=108 | hours=0.020 | epsilon=0.396
2023/03/20 11:32:49 PM > ep 77 done. total_steps=22483 | reward=4.0 | episode_steps=224 | hours=0.020 | epsilon=0.392
2023/03/20 11:32:49 PM > ep 78 done. total_steps=22545 | reward=2.0 | episode_steps=62 | hours=0.020 | epsilon=0.392
2023/03/20 11:32:49 PM > ep 79 done. total_steps=22625 | 

2023/03/20 11:33:29 PM > ep 141 done. total_steps=35297 | reward=15.0 | episode_steps=127 | hours=0.032 | epsilon=0.254
2023/03/20 11:33:29 PM > ep 142 done. total_steps=35332 | reward=2.0 | episode_steps=35 | hours=0.032 | epsilon=0.254
2023/03/20 11:33:30 PM > ep 143 done. total_steps=35476 | reward=2.0 | episode_steps=144 | hours=0.032 | epsilon=0.253
2023/03/20 11:33:30 PM > ep 144 done. total_steps=35687 | reward=12.0 | episode_steps=211 | hours=0.032 | epsilon=0.251
2023/03/20 11:33:31 PM > ep 145 done. total_steps=35835 | reward=19.0 | episode_steps=148 | hours=0.032 | epsilon=0.250
2023/03/20 11:33:32 PM > ep 146 done. total_steps=36096 | reward=11.0 | episode_steps=261 | hours=0.032 | epsilon=0.248
2023/03/20 11:33:32 PM > ep 147 done. total_steps=36221 | reward=3.0 | episode_steps=125 | hours=0.032 | epsilon=0.247
2023/03/20 11:33:32 PM > ep 148 done. total_steps=36307 | reward=7.0 | episode_steps=86 | hours=0.033 | epsilon=0.247
2023/03/20 11:33:34 PM > ep 149 done. total_st

2023/03/20 11:34:03 PM > ep 210 done. total_steps=45507 | reward=6.0 | episode_steps=28 | hours=0.041 | epsilon=0.192
2023/03/20 11:34:03 PM > ep 211 done. total_steps=45567 | reward=13.0 | episode_steps=60 | hours=0.041 | epsilon=0.192
2023/03/20 11:34:03 PM > ep 212 done. total_steps=45672 | reward=34.0 | episode_steps=105 | hours=0.041 | epsilon=0.192
2023/03/20 11:34:04 PM > ep 213 done. total_steps=45812 | reward=1.0 | episode_steps=140 | hours=0.041 | epsilon=0.191
2023/03/20 11:34:04 PM > ep 214 done. total_steps=45911 | reward=27.0 | episode_steps=99 | hours=0.041 | epsilon=0.191
2023/03/20 11:34:05 PM > ep 215 done. total_steps=46064 | reward=38.0 | episode_steps=153 | hours=0.041 | epsilon=0.190
2023/03/20 11:34:05 PM > ep 216 done. total_steps=46126 | reward=2.0 | episode_steps=62 | hours=0.042 | epsilon=0.190
2023/03/20 11:34:05 PM > ep 217 done. total_steps=46345 | reward=35.0 | episode_steps=219 | hours=0.042 | epsilon=0.189
2023/03/20 11:34:06 PM > ep 218 done. total_ste

...

2023/03/20 11:56:55 PM > ep 1675 done. total_steps=515076 | reward=1.0 | episode_steps=10 | hours=0.422 | epsilon=0.100
2023/03/20 11:56:55 PM > ep 1676 done. total_steps=515081 | reward=1.0 | episode_steps=5 | hours=0.422 | epsilon=0.100
2023/03/20 11:56:56 PM > ep 1677 done. total_steps=515371 | reward=191.0 | episode_steps=290 | hours=0.422 | epsilon=0.100
2023/03/20 11:56:56 PM > ep 1678 done. total_steps=515405 | reward=15.0 | episode_steps=34 | hours=0.422 | epsilon=0.100
2023/03/20 11:56:56 PM > ep 1679 done. total_steps=515576 | reward=143.0 | episode_steps=171 | hours=0.423 | epsilon=0.100
2023/03/20 11:56:57 PM > ep 1680 done. total_steps=515706 | reward=1.0 | episode_steps=130 | hours=0.423 | epsilon=0.100
2023/03/20 11:56:58 PM > ep 1681 done. total_steps=516048 | reward=279.0 | episode_steps=342 | hours=0.423 | epsilon=0.100
2023/03/20 11:56:59 PM > ep 1682 done. total_steps=516418 | reward=346.0 | episode_steps=370 | hours=0.423 | epsilon=0.100
2023/03/20 11:56:59 PM > ep

2023/03/20 11:57:51 PM > ep 1742 done. total_steps=535202 | reward=128.0 | episode_steps=365 | hours=0.438 | epsilon=0.100
2023/03/20 11:57:51 PM > ep 1743 done. total_steps=535486 | reward=116.0 | episode_steps=284 | hours=0.438 | epsilon=0.100
2023/03/20 11:57:54 PM > ep 1744 done. total_steps=536486 | reward=890 | episode_steps=1000 | hours=0.439 | epsilon=0.100
2023/03/20 11:57:55 PM > ep 1745 done. total_steps=536805 | reward=212.0 | episode_steps=319 | hours=0.439 | epsilon=0.100
2023/03/20 11:57:57 PM > ep 1746 done. total_steps=537371 | reward=471.0 | episode_steps=566 | hours=0.439 | epsilon=0.100
2023/03/20 11:57:57 PM > ep 1747 done. total_steps=537616 | reward=210.0 | episode_steps=245 | hours=0.439 | epsilon=0.100
2023/03/20 11:58:00 PM > ep 1748 done. total_steps=538616 | reward=900 | episode_steps=1000 | hours=0.440 | epsilon=0.100
2023/03/20 11:58:03 PM > ep 1749 done. total_steps=539616 | reward=937 | episode_steps=1000 | hours=0.441 | epsilon=0.100
2023/03/20 11:58:04

2023/03/20 11:59:34 PM > ep 1809 done. total_steps=572782 | reward=54.0 | episode_steps=82 | hours=0.466 | epsilon=0.100
2023/03/20 11:59:36 PM > ep 1810 done. total_steps=573349 | reward=510.0 | episode_steps=567 | hours=0.467 | epsilon=0.100
2023/03/20 11:59:39 PM > ep 1811 done. total_steps=574349 | reward=680 | episode_steps=1000 | hours=0.468 | epsilon=0.100
2023/03/20 11:59:39 PM > ep 1812 done. total_steps=574400 | reward=1.0 | episode_steps=51 | hours=0.468 | epsilon=0.100
2023/03/20 11:59:41 PM > ep 1813 done. total_steps=575400 | reward=779 | episode_steps=1000 | hours=0.468 | epsilon=0.100
2023/03/20 11:59:42 PM > ep 1814 done. total_steps=575658 | reward=179.0 | episode_steps=258 | hours=0.469 | epsilon=0.100
2023/03/20 11:59:43 PM > ep 1815 done. total_steps=575854 | reward=151.0 | episode_steps=196 | hours=0.469 | epsilon=0.100
2023/03/20 11:59:43 PM > ep 1816 done. total_steps=575922 | reward=41.0 | episode_steps=68 | hours=0.469 | epsilon=0.100
2023/03/20 11:59:44 PM > 

2023/03/21 12:00:40 AM > ep 1876 done. total_steps=596400 | reward=576.0 | episode_steps=705 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:41 AM > ep 1877 done. total_steps=596912 | reward=403.0 | episode_steps=512 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:42 AM > ep 1878 done. total_steps=597002 | reward=74.0 | episode_steps=90 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:42 AM > ep 1879 done. total_steps=597084 | reward=17.0 | episode_steps=82 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:42 AM > ep 1880 done. total_steps=597150 | reward=43.0 | episode_steps=66 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:43 AM > ep 1881 done. total_steps=597334 | reward=96.0 | episode_steps=184 | hours=0.485 | epsilon=0.100
2023/03/21 12:00:43 AM > ep 1882 done. total_steps=597505 | reward=148.0 | episode_steps=171 | hours=0.486 | epsilon=0.100
2023/03/21 12:00:43 AM > ep 1883 done. total_steps=597653 | reward=60.0 | episode_steps=148 | hours=0.486 | epsilon=0.100
2023/03/21 12:00:44 AM >

2023/03/21 12:01:42 AM > ep 1944 done. total_steps=618804 | reward=79.0 | episode_steps=91 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:42 AM > ep 1945 done. total_steps=618961 | reward=27.0 | episode_steps=157 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:43 AM > ep 1946 done. total_steps=619036 | reward=8.0 | episode_steps=75 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:43 AM > ep 1947 done. total_steps=619129 | reward=81.0 | episode_steps=93 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:43 AM > ep 1948 done. total_steps=619140 | reward=1.0 | episode_steps=11 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:44 AM > ep 1949 done. total_steps=619409 | reward=248.0 | episode_steps=269 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:44 AM > ep 1950 done. total_steps=619561 | reward=14.0 | episode_steps=152 | hours=0.502 | epsilon=0.100
2023/03/21 12:01:46 AM > ep 1951 done. total_steps=620152 | reward=479.0 | episode_steps=591 | hours=0.503 | epsilon=0.100
2023/03/21 12:01:48 AM > ep 

#### Loading

In [5]:
args = Namespace(batch_size=32, cuda=True, entropy_reg=0.01, env=ENV, 
                 epsilon_decay=20000, epsilon_min=0.1, epsilon_start=1.0, exp=None, 
                 frame_skip=4, freeze_interval=200, gamma=0.99, learning_rate=0.0005, 
                 logdir='runs', max_history=10000, max_steps_ep=18000, max_steps_total=4000000, 
                 num_options=4, optimal_eps=0.05, seed=0, switch_goal=True, temp=1, 
                 termination_reg=0.01, update_frequency=4, render=False, name="Not LTL")


## Instantiate Env
env, is_atari = make_env(ENV, 0, True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
devide='cpu'
## Instantiate Model
option_critic = OptionCriticFeatures(
        in_features=env.observation_space.shape[0],
        num_actions=env.action_space.n,
        num_options=args.num_options,
        temperature=args.temp,
        eps_start=0,
        eps_min=0,
        eps_decay=0,
        eps_test=0,
        device=device
    )

## Load model params
params = torch.load("models/Standard_4_0_1k", map_location=torch.device('cpu'))
option_critic.load_state_dict(params["model_params"])
option_critic.eval()

## Load Env goal
env.init_states = list(env.tocell.keys())
env.goal = {}
env.switch_goal(params["goal_state"])
print("Goal State: {}".format(params["goal_state"]))

Goal State: 62


In [6]:
params["goal_state"]

62

## Policy Graphs

In [16]:
env_states = 104
env_spec = 3
            
for i in range(env_spec):
    
    fig, ax = plt.subplots(figsize=(15, 15))    
#     ax.set_title('State {}'.format(i))
    
    # Plot Grid
    matrice = ax.matshow(env.occupancy, vmin=0, vmax=1)

    # Annulus
    if ENV=="ltl_fourrooms":
        goal_pos = [list(env.tocell[key]) for key in env.goal]
    else:
        goal_pos = [list(env.tocell[env.goal])]
    
    for pos in goal_pos:
        pos.reverse()
    for pos in goal_pos:
        annulus = patches.Annulus(pos, r=0.25, width=0.01, color='red')
        ax.add_patch(annulus)
    
    for j in range(env_states):
        env_obs = torch.zeros(env_states)
        spec_obs[i] = 1
        
        if ENV=="ltl_fourrooms":
            spec_obs = torch.zeros(env_spec)
            env_obs[j] = 1
            obs = torch.concat([env_obs, spec_obs])
        else:
            obs = env_obs
        
        state = option_critic.get_state(to_tensor(obs))
        greedy_option  = option_critic.greedy_option(state)
        action, logp, entropy = option_critic.get_action(state, greedy_option)
        
        pos = list(env.tocell[j])
        pos.reverse()
        
        dir_ = env.directions[action] 
        
        arrow = patches.Arrow(pos[0], pos[1], dir_[0]*0.3, dir_[1]*0.3, width=0.25, color='green')
        ax.add_patch(arrow)
        if ENV=="ltl_fourrooms":
            goal = {env.goal[key][0]: env.tocell[key]  for key in env.goal}
            ax.set_title("{}. {}".format(i, goal))

    plt.show()

In [ ]:
env.

In [13]:
goal_pos

[[9, 7]]

## Run

In [ ]:
## Instantiate Env
env, is_atari = make_env("fourrooms", True)
device = torch.device('cuda' if torch.cuda.is_available() and args.cuda else 'cpu')
device='cpu'
## Instantiate Model
option_critic = OptionCriticFeatures(
        in_features=env.observation_space.shape[0],
        num_actions=env.action_space.n,
        num_options=args.num_options,
        temperature=args.temp,
        eps_start=0,
        eps_min=0,
        eps_decay=0,
        eps_test=0,
        device=device
    )

## Load model params
params = torch.load("models/Standard_4_0_1k", map_location=torch.device('cpu'))
option_critic.load_state_dict(params["model_params"])
option_critic.eval()

## Load Env goal
env.init_states = list(env.tocell.keys())
env.goal = {}
env.switch_goal(params["goal_state"])

obs   = env.reset()
state = option_critic.get_state(to_tensor(obs))
current_option = option_critic.greedy_option(state)
rewards = 0
while True:        
    
    action, logp, entropy = option_critic.get_action(state, current_option)
    next_obs, reward, done, _ = env.step(action)
    rewards += reward
    
    state = option_critic.get_state(to_tensor(next_obs))
    option_termination, current_option = option_critic.predict_option_termination(state, current_option)

    if done:
        obs   = env.reset()
        print(rewards)
        rewards = 0
#         break

#     logger.log_data(reward)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
